In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import hyperspy.api as hs
from hyperspy.signals import Signal1D
from snmfem.estimators import NMF
from snmfem.models.edxs import G_EDXS

# Dataset creation (optional)

In [ ]:

spim = create_dataset()

# save to hspy
hs.save("filename.hspy", spim)



# Data loading and problem definition

In [ ]:

spim = hs.load("filename.hspy")


In [ ]:
generate_synthetic_datasets() 

spim_A = load_dataset_A(sample=0) # Hard coded configuration in the code
spim_B = load_dataset_B()
spim_C = load_dataset_C()
spim_real = load_dataset_real()


# Problem solving

In [ ]:

# interface semi-black box
param_opt = dict(...)
est = NMF(param_opt)
# problem_type: "nothing", "no_bremsstrahlung", "bremsstralung", "updated_bremsstrahlung"
G = spim.extract_G(problem_type)
spim.decomposition(est, G)
P = spim.get_P()
spim.plot_P()
spim.eval_loss()


# interface flux de données
param_opt = dict(...)
est = NMF(param_opt)
X, G, true_P, true_A = spim.extract(problem_type)

sol_P, sol_A = est.fit_transform(X, G)
# sol_P, sol_A = est.fit_transform(X, G, true_P, true_A)
eval_loss(sol_P,true_P)





In [ ]:
class EdxsSignal1D(Signal1D):
    def extract_metadata(self, problem):
        d = {}
        d["shape_2d"] = self.metadata["shape_2d"]
        # true_D = None
        # true_A = None
        d["model_params"] = self.metadata["model_params"]
        # g_params = self.metadata["g_params"]
        if problem == "update_bremsstrahlung":
            G = self.update_G
        else:
            G = None
        return G, d

    def extract(self):

        metadata = convert_metadata(self)
        return G, X, metadata

    def extract_true(self):
        return 0

    def decomposition_edxs(self, algorithm, **kwargs):
        G, X, metadata = self.extract()
        GP, A = super(self).decomposition(algorithm, metadata)
        # reassignment
        self.get_P(recompute=True)

    def get_P(self, recompute=False):
        if (self.P is None) or recompute:
            self.P = np.linalg.lstsq(self.G, self.phase, rcond=None)
        return self.P

    def update_G(P=None, G=None):
        model_params = self.metadata["model_params"]
        g_params = self.metadata["g_params"]
        G = G_EDXS(model_params, g_params, P=P, G=G)
        return G

    def plot_P(self):
        # beautiful plot

    @property
    def true_A(self):
        return self.metadata["true_A"]


# Evaluation